In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy
from collections import defaultdict

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.meta_pgd import *
from src.attacks.greedy_gd import *
print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Cora')
data = cora_dataset[0].to(device)
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [4]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [64]).to(device)

In [5]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 1.9408074617385864, Val Loss - 1.809523582458496, Val Accuracy - 0.677
Epoch 20, Train Loss - 0.014577998779714108, Val Loss - 0.6161941289901733, Val Accuracy - 0.797
Epoch 40, Train Loss - 0.006608617026358843, Val Loss - 0.6547042727470398, Val Accuracy - 0.798
Epoch 60, Train Loss - 0.011164320632815361, Val Loss - 0.6110913157463074, Val Accuracy - 0.808
Epoch 80, Train Loss - 0.014906632713973522, Val Loss - 0.6338757276535034, Val Accuracy - 0.801
Epoch 100, Train Loss - 0.012234837748110294, Val Loss - 0.6280432939529419, Val Accuracy - 0.799
Epoch 120, Train Loss - 0.011410451494157314, Val Loss - 0.611633837223053, Val Accuracy - 0.816
Epoch 140, Train Loss - 0.010603055357933044, Val Loss - 0.6338784098625183, Val Accuracy - 0.804
Epoch 160, Train Loss - 0.012238751165568829, Val Loss - 0.6285136938095093, Val Accuracy - 0.811
Epoch 180, Train Loss - 0.009514456614851952, Val Loss - 0.6190340518951416, Val Accuracy - 0.816
Epoch 200, Train Loss - 0.0106

In [6]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.804
Initial Loss: 0.644078254699707


In [14]:
attacker.num_budgets

NameError: name 'attacker' is not defined

In [7]:
from collections import defaultdict 
from torch_geometric.utils import to_networkx

# Run MetaPGD attack
ptb_rate = 1.0  # 100% perturbation rate
attacker = MetaPGD(data, device=device)
attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask,
                         lambda_=0.)
attacker.reset_parameters(seed=42)
attacker.reset()

best_loss, best_pert = attacker.attack(ptb_rate, base_lr=0.01, grad_clip=1, iterations=400)

Running PGD Attack...:   0%|          | 0/400 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, device='cuda:0')
5278 tensor(5077, de

In [8]:
# Extract edges from the perturbation matrix
edges_added = []
modified_adj = attacker.get_perturbed_adj(best_pert).cpu().numpy()
original_adj = attacker.adj.cpu().numpy()

# Find added edges (in the upper triangle to avoid duplicates since graph is undirected)
for i in range(data.num_nodes):
    for j in range(i+1, data.num_nodes):  # Upper triangle only
        if modified_adj[i, j] > 0 and original_adj[i, j] == 0:
            edges_added.append((i, j))

print(f"Total edges added: {len(edges_added)}")

Total edges added: 5076


In [11]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.20
budget = int(ptb_rate * initial_edge_count)
print(f"initial edges: {initial_edge_count}")
print(f"Budget: {budget}")

initial edges: 2639
Budget: 527


In [15]:
print("Unique perturbed edges (undirected):", len(set(frozenset(e) for e in all_perturbed_edges)))

Unique perturbed edges (undirected): 1814


In [12]:
attacker.num_budgets

5278

In [13]:
attacker.adj_changes.sum()

tensor(10154., device='cuda:0')

In [14]:
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import networkx as nx

In [15]:
print(len(edges_added))
print(budget)

5076
527


In [16]:
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(edges_added):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = edges_added[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = Metattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [17]:
splits = [0, 0.5]
split_dic = defaultdict(list)

In [18]:
for s in splits:
    print(s)
    split_dic[s] = two_phase_attack(s)

0

--- Running split: 0 ---
Second phase perturbation rate: 0.1000
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/527 [00:00<?, ?it/s]

Final Accuracy: 0.7990
Accuracy Drop: 0.0050
0.5

--- Running split: 0.5 ---
Second phase perturbation rate: 0.0500
Phase 1: Added 263 edges out of requested 263.


Peturbing graph...:   0%|          | 0/277 [00:00<?, ?it/s]

Final Accuracy: 0.7920
Accuracy Drop: 0.0120
